# MicroStrateg(P)ython Migration

## Load libraries

In [1]:
import json
from mstrio.object_management import folder
import pandas as pd
from time import sleep
import shutil
import os
from mstrio.connection import Connection
from mstrio.object_management.migration import (
    PackageType,
    Migration,
    PackageConfig,
    PackageSettings
)

from mstrio.object_management.migration.package import (
    Action,
    ImportStatus,
    PackageStatus,
    ValidationStatus,
)



def bld_mig_list_from_sh_fold(conn, folder_id,action= "USE_EXISTING",include_dependents= False ):

    #obj_l = get_folder_obj_l(conn=conn, folder_id=folder_id)
    i_folder = folder.Folder(connection=conn, id=folder_id)
    obj_l = i_folder.get_contents(to_dictionary=True)
    
    obj_det_l=[]
    for obj_d in obj_l:
        #print(obj_d)
        if obj_d["type"] == 18:
            obj_det_d = {}
            obj_det_d["id"] = obj_d["target_info"]["id"]
            obj_det_d["type"] = obj_d["target_info"]["type"]
            obj_det_d["action"] = action
            obj_det_d["include_dependents"] = include_dependents
            obj_det_d["subtype"] = obj_d["target_info"]["subtype"] # only for read out object definition (see end of note book)
            obj_det_l.append(obj_det_d.copy())  

    return obj_det_l


In [2]:
from mstrio.connection import Connection
with open('..\\config\\user_d.json', 'r') as openfile:
    user_d = json.load(openfile)

conn_params =  user_d["conn_params"]
source_conn = Connection(**conn_params)

Connection to MicroStrategy Intelligence Server has been established.
No project selected.


## Configure Migration

In [ ]:
#define Package
usr_migration_folder_name="Daniel_Migration"
usr_package_folder_name="20250223_demo"


short_cut_folder_id="986E6D1A427030A710A0E68F74290482"

# Connect Projects
with open('..\\config\\user_d.json', 'r') as openfile:
    user_d = json.load(openfile)
conn_params =  user_d["conn_params"]
#set user credentials and open a connection to the i-server

source_conn = Connection(**conn_params)
target_conn = Connection(**conn_params)

source_project_id="B7CA92F04B9FAE8D941C3E9B7E0CD754"
target_project_id="00F3103E41FF3743A057C3B5313595B7"
package_settings = PackageSettings(
    Action.FORCE_REPLACE,
    PackageSettings.UpdateSchema.RECAL_TABLE_LOGICAL_SIZE,
    PackageSettings.AclOnReplacingObjects.REPLACE,
    PackageSettings.AclOnNewObjects.KEEP_ACL_AS_SOURCE_OBJECT,
)

## Connect to projects

In [ ]:
def bld_full_mig_fold(conn, mig_fold_str):
    mstr_mig_folder=conn.environment.storage_service.location 
    full_mig_fold_str= conn.environment.storage_service.location  +"\\"+ mig_fold_str + "\\"
    return full_mig_fold_str

conn_params["project_id"]=source_project_id
source_conn = Connection(**conn_params)

conn_params["project_id"]=target_project_id
target_conn = Connection(**conn_params)

package_name=usr_package_folder_name
mig_fold_str=usr_migration_folder_name +"\\"+ usr_package_folder_name

bld_full_mig_fold(conn=source_conn,mig_fold_str=mig_fold_str)

In [ ]:
package_content_info=bld_mig_list_from_sh_fold(conn=source_conn, folder_id=short_cut_folder_id,action= "FORCE_REPLACE",include_dependents= True)
package_config = PackageConfig(
    package_settings,  package_content_info # [ package_content_from_object]
)

my_obj_mig = Migration.create_object_migration(
    connection=source_conn,
    toc_view=package_config,
    name=package_name,
    project_id=source_conn.project_id,
)

while my_obj_mig.package_info.status == PackageStatus.CREATING:
    sleep(2)
    my_obj_mig.fetch()

my_obj_mig.package_info.status

## Check Package

In [ ]:
mmp_file = "D:\\shared_drive\MSTR_Migrations\\Daniel_Migration\\20250223_demo\\A71B5205050B48D58C9EA8F30C81BE6E.mmp"
target_project_id = target_conn.project_id
source_project_id = source_conn.project_id

package_type="project"

from_file_mig = Migration.migrate_from_file(
    connection=target_conn,
    file_path=mmp_file,
    package_type=package_type,
    name="mmp_id_" + mmp_file[-36:][:32],
    target_project_id=source_project_id
)
from_file_mig.project_id=source_project_id
"""
from_file_mig.trigger_validation(
    target_env=target_conn, target_project_name=target_conn.project_name
)

while from_file_mig.validation.status == ValidationStatus.VALIDATING:
    sleep(2)
    from_file_mig.fetch()


print(my_obj_mig.validation)
"""

In [ ]:
target_conn.headers["Prefer"]="respond-async"

In [ ]:
url='http://85.214.60.83:8080/MicroStrategyLibrary/api/migrations/8E1844846E444577AA716852AC9E2C86:0E9D611FE2E34421982A3D455DA9E9ED/validation'
rep=target_conn.put(url=url, data=json.dumps(body))
rep

In [ ]:
rep.text

## Validate Package

In [ ]:
# Check migration validation status
def trigger_validation_WS(my_obj_mig,target_conn):
    my_obj_mig.trigger_validation(
        target_env=target_conn, target_project_name=target_conn.project_name
    )
    
    while my_obj_mig.validation.status == ValidationStatus.VALIDATING:
        sleep(2)
        my_obj_mig.fetch()
    return my_obj_mig

my_obj_mig=trigger_validation_WS(my_obj_mig,target_conn)
print(my_obj_mig.validation)

## Move Migration Pack to migration folder

In [ ]:
package_file = mstr_mig_folder +"/" +my_obj_mig.package_info.storage.path
package_file=os.path.normpath(package_file)
my_obj_mig_path=shutil.move(package_file, migration_folder)

print("org_file: " +package_file)
print("migration folder: " +my_obj_mig_path)


## Trigger migration

In [ ]:
my_obj_mig_path = "D:\\shared_drive\MSTR_Migrations\\Daniel_Migration\\20250223_demo\\A71B5205050B48D58C9EA8F30C81BE6E.mmp"
from_file_mig = Migration.migrate_from_file(
    connection=target_conn,
    file_path=my_obj_mig_path,
    package_type=PackageType.OBJECT,
    name="object_mig_from_local_storage",
    target_project_name=target_conn.project_name,
)

while from_file_mig.import_info.status == ImportStatus.IMPORTING:
    sleep(2)
    from_file_mig.fetch()

from_file_mig.import_info.status

In [ ]:
from_file_mig.import_info

## Archive Migration

### move undo mmp file to migration folder

In [ ]:
## move undo mmp to migration folder
undo_source_file = mstr_mig_folder +"/" +from_file_mig.import_info.undo_storage.path
package_file=os.path.normpath(package_file)
new_undo_file=shutil.move(undo_source_file, migration_folder+"Undo.mmp")

print("org_undo_file: " +undo_source_file)
print("migration folder: " +new_undo_file)

### store migrated objects to CSV file

In [ ]:
package_content_df=pd.DataFrame(package_content_info)
package_content_df.to_csv(migration_folder+"package_content.csv")
package_content_df

### save package & migration files

In [ ]:
file_path=migration_folder+"package_info.txt"
with open(file_path, "w", encoding="utf-8") as file:
    file.write(str(my_obj_mig.package_info))

file_path=migration_folder+"migration_info.txt"
with open(file_path, "w", encoding="utf-8") as file:
    file.write(str(from_file_mig.import_info))


## Read & Store Object definition

In [ ]:
from mstr_robotics.read_out_prj_obj import read_gen
i_read_gen=read_gen()

obj_def_l=[]
for obj in package_content_info:
    obj_def=i_read_gen.get_obj_def(conn=source_conn,object_id=obj["id"],obj_type=obj["type"],obj_sub_type=obj["subtype"])
    obj_def_l.append(obj_def)

file_path=migration_folder+"object_def.json"
with open(file_path, "w", encoding="utf-8") as file:
    file.write(json.dumps(obj_def_l))
    
len(obj_def_l)

In [ ]:
obj_def_l[0]

In [ ]:
source_conn.close()
target_conn.close()

## Run undo

In [ ]:
from_file_mig = Migration.migrate_from_file(
    connection=target_conn,
    file_path=new_undo_file,
    package_type=PackageType.OBJECT,
    name="object_mig_from_local_storage",
    target_project_name=target_conn.project_name,
)

while from_file_mig.import_info.status == ImportStatus.IMPORTING:
    sleep(2)
    from_file_mig.fetch()

from_file_mig.import_info.status

In [ ]:
url=f"{source_conn.base_url}/api/migrations?offset=0&limit=1000"
mig_d_l=source_conn.get(url).json()
mig_d_l

In [ ]:
url=f"{source_conn.base_url}/api/migrations?offset=0&limit=1000"
mig_d_l=source_conn.get(url).json()
for m in mig_d_l["data"]:
    p=source_conn.delete(f'http://85.214.60.83:8080/MicroStrategyLibrary/api/migrations?packageId={m["packageInfo"]["id"]}')
    print(p)

In [ ]:
mig_d_l

In [ ]:
url=f"{source_conn.base_url}/api/migrations?offset=0&limit=1000"
mig_d_l=source_conn.get(url).json()
mig_d_l

In [ ]:
conn_params

In [ ]:
source_conn.get('http://85.214.60.83:8080/MicroStrategyLibrary/api/users/54F3D26011D2896560009A8E67019608').json()

In [ ]:
source_conn.get('http://85.214.60.83:8080/MicroStrategyLibrary/api/usergroups/33B30CDE48BD2264BFB394AC5D606519/privileges?privilege.level=project&projectId=40DA7A1549651FD60A9D39AAB7EC77B0' ).json()

In [4]:
def bld_proj_usg_priv_d_l(proj_id,usg_id,proj_usg_priv_d_l,all_privilliges_d_l):
    u_p_priv_d_l=[]
    u_p_priv_d={"project_id":proj_id,"user_grp_id":usg_id}
    for u_p_priv in proj_usg_priv_d_l:
        
        for priv in all_privilliges_d_l:

            if u_p_priv["privilege"]["id"]==priv["id"]:
                u_p_priv_d["id"]=priv["id"]
                u_p_priv_d["name"]=priv["name"]
                u_p_priv_d_l.append(u_p_priv_d.copy())
    return u_p_priv_d_l


def read_usg_proj_priv(conn,proj_id,usg_id):
        proj_usg_priv_d_l=conn.get(f'{source_conn.base_url}/api/usergroups/{usg_id}/privileges?privilege.level=project&projectId={proj_id}' ).json()
        #print(proj_usg_priv_d_l)
        return proj_usg_priv_d_l

    


In [5]:
conn=source_conn
all_privilliges_d_l=conn.get(f'{conn.base_url}/api/iserver/privileges').json()
all_project_d_l=conn.get(f'{conn.base_url}/api/projects').json()
user_group_l = conn.get(f'{conn.base_url}/api/usergroups?offset=0&limit=-1').json()
all_u_p_priv_d_l=[]
for usg_id in user_group_l:
    for p in all_project_d_l:
        proj_usg_priv_d_l=read_usg_proj_priv(conn=conn,proj_id=p["id"],usg_id=usg_id["id"])
        if len(proj_usg_priv_d_l["privileges"]):
            u_p_priv_d_l=bld_proj_usg_priv_d_l(proj_id=p["id"],usg_id=usg_id["id"],
                                               proj_usg_priv_d_l=proj_usg_priv_d_l["privileges"],all_privilliges_d_l=all_privilliges_d_l)
            all_u_p_priv_d_l.extend(u_p_priv_d_l.copy())
                                               
        
df=pd.DataFrame(all_u_p_priv_d_l)      
df

,project_id,user_grp_id,id,name
0,B7CA92F04B9FAE8D941C3E9B7E0CD754,98AA5CA04DD6D20C6875C6B314874A1C,63,Use Report Objects Window
1,B7CA92F04B9FAE8D941C3E9B7E0CD754,98AA5CA04DD6D20C6875C6B314874A1C,67,Use View Filter Editor
2,B7CA92F04B9FAE8D941C3E9B7E0CD754,98AA5CA04DD6D20C6875C6B314874A1C,68,Create Derived Metrics
3,B7CA92F04B9FAE8D941C3E9B7E0CD754,98AA5CA04DD6D20C6875C6B314874A1C,71,Web create Derived Metrics and Derived Attributes
4,B7CA92F04B9FAE8D941C3E9B7E0CD754,98AA5CA04DD6D20C6875C6B314874A1C,74,Web use Report Objects Window
...,...,...,...,...
3825,40DA7A1549651FD60A9D39AAB7EC77B0,85A05F10986F11D48CD100B0D02A75D0,40,Web format Grid and Graph
3826,40DA7A1549651FD60A9D39AAB7EC77B0,85A05F10986F11D48CD100B0D02A75D0,41,Web subscribe to History List
3827,40DA7A1549651FD60A9D39AAB7EC77B0,85A05F10986F11D48CD100B0D02A75D0,49,Subscribe to Email
3828,40DA7A1549651FD60A9D39AAB7EC77B0,85A05F10986F11D48CD100B0D02A75D0,265,Use Workstation


In [5]:
from mstrio.project_objects.report import Report
from mstrio.api import reports
conn=source_conn
def open_Instance(conn, report_id):
    rep_instance = reports.report_instance(connection=conn, report_id=report_id)
    return rep_instance.json()["instanceId"]

def report_df(conn, report_id, instance_id):
    i_Report = Report(connection=conn, id=report_id, instance_id=instance_id)
    return i_Report.to_dataframe()


conn.select_project("40DA7A1549651FD60A9D39AAB7EC77B0")
rep_usr_prj_action_id="54AC818944D345BA7D050CB9C86B0D06"

instance_id=open_Instance(conn=conn, report_id=rep_usr_prj_action_id)
pa_report_df=report_df(conn=conn, report_id=rep_usr_prj_action_id, instance_id=instance_id)

pa_report_df


Report object named: 'UserActions' with ID: '54AC818944D345BA7D050CB9C86B0D06'


,Metadata Account,User Status,Action Category,Action Type,User,Project,Sessions
0,15394FF445A2FB19A6993B8043828C53:54F3D26011D2896560009A8E67019608,Enabled,Modify History List Messages,Create History List Message,Administrator,MicroStrategy Tutorial,1
1,15394FF445A2FB19A6993B8043828C53:54F3D26011D2896560009A8E67019608,Enabled,Cube Executions,Cube Publish,Administrator,MicroStrategy Tutorial,53
2,15394FF445A2FB19A6993B8043828C53:54F3D26011D2896560009A8E67019608,Enabled,Cube Executions,Cube Publish,Administrator,MicroStrategy Tutorial PreProd,1
3,15394FF445A2FB19A6993B8043828C53:54F3D26011D2896560009A8E67019608,Enabled,Cube Executions,Cube Publish,Administrator,Platform Analytics,1172
4,15394FF445A2FB19A6993B8043828C53:54F3D26011D2896560009A8E67019608,Enabled,Cache Modifications,Delete Report Cache,Administrator,MicroStrategy Tutorial,3
5,15394FF445A2FB19A6993B8043828C53:54F3D26011D2896560009A8E67019608,Enabled,Cache Modifications,Delete Report Cache,Administrator,Platform Analytics,3
6,15394FF445A2FB19A6993B8043828C53:54F3D26011D2896560009A8E67019608,Enabled,Cache Modifications,Delete Report Cache,Administrator,Unknown,1
7,15394FF445A2FB19A6993B8043828C53:54F3D26011D2896560009A8E67019608,Enabled,Edit,Edit in Design Mode,Administrator,MicroStrategy Tutorial,165
8,15394FF445A2FB19A6993B8043828C53:54F3D26011D2896560009A8E67019608,Enabled,Edit,Edit in Design Mode,Administrator,MicroStrategy Tutorial PreProd,1
9,15394FF445A2FB19A6993B8043828C53:54F3D26011D2896560009A8E67019608,Enabled,Edit,Edit in Design Mode,Administrator,Platform Analytics,16
